<a href="https://colab.research.google.com/github/nicolascavalli/KaggleProjects/blob/main/MultipleDiseaseClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'multiple-disease-prediction:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4532617%2F7752217%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240317%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240317T140750Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3dcf983b49a41c193ee2c9d17d1db207280a35bcfc5394074e15505b27987870c4872c57746276504cb208b1ffced528e5fe87317622283960ceb1f648136261dd202f1a40d0ef3178e094c56b5b513be62d9776372538acb22be7433779938a63c7dcb06d255ad872ceb45db3c69f13edaf140c32ad1f2449b6148c5a210bb07391e227c0a8db9e422cc8a0c271b34bc06854dd79d5959b359b38bf1e990dc8122015e19a978298fcc7b7dee89b0d1382c07da516cc5e998ebb8fc4cf4a16e5d34a621e5f4f4909e0cd0e7aebab0b980013cac9d08a503d35dc201cbd3c358ebbc1d0a6e1e80944e866ea98bd03283d217ad80c896dcd323828236554f57428'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### First Impressions on Dataset

In [ ]:
df = pd.read_csv('/kaggle/input/multiple-disease-prediction/blood_samples_dataset_test.csv')
df.head()

In [ ]:
#Analysing data types
df.info()

As you can see, there aren't missing values in dataset and all the atribbutes are float values.

In [ ]:
#Verifying the shape
df.shape

In [ ]:
#Verifying if there's duplicate
df.drop_duplicates(inplace=True)
df.shape

There aren't duplicates rows on dataset

### Exploratory Data Analysis

In [ ]:
df.describe()

Notice that there are some negative values in certain features. we need to normalize the dataset

In [ ]:
#Analysing the distribution
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt

df.plot.box()
plt.xticks(rotation=60, ha='right');

It seems that there aren't outliers

In [ ]:
#Analysing the distribution of the target feature
df['Disease'].value_counts(normalize=True)

In [ ]:
#Applying LabelEncoder in the target to verify the correlation between features
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Disease'] = le.fit_transform(df['Disease'])
df.head()

In [ ]:
#Verifying the correlation
import seaborn as sns
fig, ax = plt.subplots(figsize=(20, 20))
sns.heatmap(df.corr(), annot=True, ax=ax)

In [ ]:
sns.scatterplot(data=df, x='Glucose', y='Troponin', hue='Disease')

In [ ]:
sns.scatterplot(data=df, x='Mean Corpuscular Volume', y='Cholesterol', hue='Disease')

In [ ]:
#Normalize the dataset and split in atributtes and target

from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X = pd.DataFrame(mms.fit_transform(df.drop(columns='Disease')), index=df.drop(columns='Disease').index, columns=df.drop(columns='Disease').columns)
X.head()


In [ ]:
#Splitting dataset in train and test
y = df['Disease']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)

In [ ]:
y

### Modeling

#### Logistic Regression

In [ ]:
#Fitting model to train dataset
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=500, multi_class='multinomial')
lr.fit(X_train, y_train)

In [ ]:
#Predicting the test dataset
lr_pred = lr.predict(X_test)

In [ ]:
#Verifying metrics
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score

In [ ]:
print(f'Accuracy: {accuracy_score(y_test, lr_pred)}')
print(f'Balanced Accuracy: {balanced_accuracy_score(y_test, lr_pred)}')

#### Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
dtc_pred = dtc.predict(X_test)

In [ ]:
print(f'Accuracy: {accuracy_score(y_test, dtc_pred)}')
print(f'Balanced Accuracy: {balanced_accuracy_score(y_test, dtc_pred)}')



#### SVM

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc = SVC()
svc.fit(X_train, y_train)
svc_pred = svc.predict(X_test)

In [ ]:
print(f'Accuracy: {accuracy_score(y_test, svc_pred)}')
print(f'Balanced Accuracy: {balanced_accuracy_score(y_test, svc_pred)}')

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)

In [ ]:
print(f'Accuracy: {accuracy_score(y_test, rfc_pred)}')
print(f'Balanced Accuracy: {balanced_accuracy_score(y_test, rfc_pred)}')

So, after testing several models, decision tree classifier is the one that performed better.